<div style="width:100%;height:125px;text-align:center;border: 4px solid #F2CA03;background-color:#F2CA03;color:black">

<header style="width:100%;height:140px;">
  <h1>Chapter 8.2</h1>
    <h1><b>l1,l2 regularization - part 3</b></h1>
</header>

<div> 

# brief contents:

<div style="position: relative;height:100px;">
    
<div  style="width:300px;position:absolute;left: auto;border: 4px solid white;background-color:#F2CA03;color:white">
    <header></header>
    <a href="#Forward_propagation" style="padding:5px;color:black;text-align: center;" >
      <h4 ><b>Dropout Regularization</b></h4>
      </a>
 </div>
    
 <div style="width:300px;position:absolute;left: 305px;border: 4px solid white;background-color:#F2CA03;color:whitee">
    <header></header>
    <a href="#Data"style="position: relative;padding:5px;color:black;text-align: center;" >
      <h4 ><b>Data</b></h4>
      </a>
     
  </div>
    <div style="width:300px;position:absolute;left: 610px;border: 4px solid white;background-color:#F2CA03;color:white">
    <header></header>
    <a href="#model"style="position: relative;padding:5px;color:black;text-align: center;" >
      <h4 ><b>model adn feed</b></h4>
      </a>
        </div>

    
   <div  style="width:300px;position:absolute;left: 915px;border: 4px solid white;background-color:#F2CA03;color:white">
    <header></header>
    <a href="#list"style="position: relative;padding:5px;color:black;text-align: center;">
      <h4 ><b></b></h4>
      </a>
    </div>
  
    
   <div  style="width:300px;position:absolute;left: 1220px;border: 4px solid white;background-color:#F2CA03;color:white">
    <header></header>
    <a href="#list"style="position: relative;padding:5px;color:black;text-align: center;" >
      <h4 ><b></b></h4>
      </a>
    </div>
</div>

<!------------------------------------------------------------------------------------------------------------------------>

<div id="Dropout_Regularization" style="width:100%;height:70px;border: 4px solid #F2CA03;background-color:#F2CA03;color:white;text-align:center;border-radius: 25px;padding:3px;color:black;" ><h1>Dropout Regularization<h1></div>


<div style="border: 4px solid #BFE6FF;background-color:#BFE6FF;color:black;border-radius: 5px;padding:7px">
  <strong> Refrence: </strong><br>
    


paper: https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf
    
</div>

The only difference is that the dropout layers will periodically drop some of their neurons during training. You can use dropout layers on regular feedforward neural networks. 

The usual hyper-parameters for a dropout layer are the following:
* Neuron Count
* Activation Function
* Dropout Probability

The neuron count and activation function hyper-parameters work exactly the same way as their corresponding parameters in the dense layer type mentioned previously. The neuron count simply specifies the number of neurons in the dropout layer. The dropout probability indicates the likelihood of a neuron dropping out during the training iteration. Just as it does for a dense layer, the program specifies an activation function for the dropout layer.

<img style="width:200px" src="https://camo.githubusercontent.com/3afce3daa906c32c976682e6fb9499c979176a94/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f6a656666686561746f6e2f7438315f3535385f646565705f6c6561726e696e672f6d61737465722f696d616765732f636c6173735f395f64726f706f75742e706e67">

Animation that shows how [dropout works](https://yusugomori.com/projects/deep-learning/dropout-relu) .

A certain percentage of neurons will be masked during each training step. All neurons return after training is complete. To make use of dropout in Keras use the Dropout layer and specify a dropout probability. This is the percent of neurons to be dropped. Typically, this is a low value, such as 0.1.

<div id="DATA" style="width:100%;height:70px;border: 4px solid #F2CA03;background-color:#F2CA03;color:white;text-align:center;border-radius: 25px;padding:3px;color:black;" ><h1>DATA<h1></div>

In [1]:
import pandas as pd
df = pd.read_csv("data/auto-mpg.csv")
df.sample()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
232,16.0,8,351.0,149,4335,14.5,77,1,ford thunderbird


In [9]:
import pandas as pd
from farhad_DL.utility import missing_median, to_xy
from sklearn.model_selection import train_test_split
df = pd.read_csv("data/auto-mpg.csv", na_values=["?","NA"])
df.drop("name",axis=1, inplace=True)
missing_median(df,"horsepower")
x, y = to_xy(df, 'mpg')

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25, random_state=42)


<div id="model" style="width:100%;height:70px;border: 4px solid #F2CA03;background-color:#F2CA03;color:white;text-align:center;border-radius: 25px;padding:3px;color:black;" >
    <h1>model and feeed<h1></div>


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.logging import set_verbosity, ERROR

from tensorflow.keras import regularizers 

from sklearn.metrics import confusion_matrix, mean_squared_error
from farhad_DL.utils import TimeSummary, plot_training_summary, Estimate_Deeptime

import numpy as np

Using TensorFlow backend.


In [17]:
def Regularization():
    model = Sequential()
    
    model.add(Dense(50, input_dim=x.shape[1], activation="relu"))
    model.add(Dropout(0.01))
    
    model.add(Dense(25,activation="relu"))
    model.add(Dense(10,kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation="relu"))
    
    
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer= 'adam', metrics=["accuracy"])
    
    return model
    
    
    

In [18]:
set_verbosity(ERROR)
epochs = 1000
monitor = EarlyStopping(monitor='val_cross',mode='auto',verbose=1,min_delta=1e-3,patience=5)
#estimate_cpu = Estimate_Deeptime(epochs)
time_summary = TimeSummary()

model = Regularization()
summary = model.fit(x_train,y_train, validation_data=(x_test,y_test), 
          callbacks=[monitor,time_summary], verbose=0,epochs=epochs)

pred = model.predict(x_test)
rm = np.sqrt(mean_squared_error(pred, y_test))
score = model.evaluate(x_test, y_test)
print("Final score (RMSE):",rm)
print("Final Losse:", score[0])
print("Final accuracy:",score[1])
#plot_training_summary(time_summary,summary)

100/100 [==============================] - 0s 49us/step
Final score (RMSE): 2.4666355
Final Losse: 18.628430099487304
Final accuracy: 0.18
